In [32]:
import mysql.connector

def sql_query(query:str):
    connection = mysql.connector.connect(
        host="gz-cdb-5scrcjb5.sql.tencentcdb.com",
        user="db",
        password="dbdb905905",
        database="sele",
        port=63432  
    )
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    formatted_result = ", ".join([str(row) for row in result])
    return formatted_result


In [150]:
def helper(issue):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") 
    role_setting = '''
    你的角色是一个协助其他ai agent解决问题的agent，你需要根据对方Agent的问题和对话log，给出合理的建议。
    告诉他问题潜在的解决方案，或者提供一些思路，帮助他解决问题。
    '''
    
    messages = [{"role": "system","content": role_setting}]
    messages.append({"role": "user","content": '对方Agent遇到的问题是：' + issue + '，请问您有什么想法？'})
    # messages.append({"role": "system","content": '以下是对方和用户的对话log'})
    # messages.append(context)
    
    response = client.chat.completions.create(
        model="glm-4",
        messages=messages,
    )
    
    return response.choices[0].message.content
    
result = helper('现在需要将数据查询的任务分解，现在完全没有任何数据信息，需要从show tables入手，然后查询最近一周的订单数据，然后计算销售额，用户询问“最近一周的销售额是多少”')

In [152]:
result

'对于这个问题，以下是我为对方Agent提供的建议和解决方案：\n\n1. **任务分解**：\n   - **步骤一：获取数据表信息**：首先使用 `SHOW TABLES` 语句获取所有数据表的列表，以了解数据库的结构和相关的表。\n   - **步骤二：确定订单数据表**：根据业务逻辑，找出存储订单数据的表。\n   - **步骤三：确定时间范围**：根据用户的问题，需要查询的是最近一周的数据。\n\n2. **查询最近一周的订单数据**：\n   - 在确定了订单数据表之后，可以使用 SQL 的 `WHERE` 子句结合日期函数来筛选出最近一周的数据。例如：\n     ```sql\n     SELECT * FROM orders\n     WHERE order_date BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 7 DAY) AND CURRENT_DATE;\n     ```\n   - **注意**：这里假设 `order_date` 是存储订单日期的字段。\n\n3. **计算销售额**：\n   - 在得到了最近一周的订单数据后，可以计算销售额。假设订单表中有一个字段是 `amount` 存储每笔订单的金额，可以使用 `SUM` 函数来计算总销售额：\n     ```sql\n     SELECT SUM(amount) AS total_sales\n     FROM orders\n     WHERE order_date BETWEEN DATE_SUB(CURRENT_DATE, INTERVAL 7 DAY) AND CURRENT_DATE;\n     ```\n\n4. **其他考虑**：\n   - **数据准确性**：确认订单数据中的金额字段是否已经包含了税费和折扣等，确保计算的销售额是准确的。\n   - **性能优化**：如果数据量很大，可以考虑对 `order_date` 字段建立索引以提高查询效率。\n\n5. **反馈给用户**：\n   - 在完成计算后，将查询结果以用户友好的方式返回，例如：“最近一周的销售额为XX元”。\n\n最后，对于这种类型的问题，建议对方Agent：\n- **与用户确认需求**：确认用户询问的“最近一周销售额”是否包括特定的产品、服

In [72]:
from zhipuai import ZhipuAI
import json

def sql_querier(input,context=None):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey

    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },
        {
        "type": "function",
        "function": {
            "name": "helper",
            "description": "无法回答用户问题时，可以调用此函数来请求帮助。",
            "parameters": {
                "type": "object",
                "properties": {
                    "issue": {
                        "description": "将遇到的问题在此详细描述。",
                        "type": "string"
                    }
                },
                "required": ["issue"]
            },
            }
        },
        ]

    role_setting = '''
    你是一个协助回答招标信息的助手。
    当你尝试回答用户问题，但是需要查询数据库时，你可以调用sql_query函数来查询数据库。
    当你无法回答问题的时候，尝试调用helper函数来请求帮助。
    依据helper函数的返回结果，你可以尝试继续用sql_query函数来查询正确的表来获取更多信息，从而回答问题
    '''
    
    messages = [{"role": "system","content": role_setting}]
    if context:
        messages.append({"role": "user","content": '用户的问题是：' + input + '。上一次查询的结果是：' + context})
    else:
        messages.append({"role": "user","content": '用户的问题是：' + input})
    
    response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    max_loop_count = 3
    loop_count = 0
    while loop_count < max_loop_count and response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        print('这是函数参数' + args)

        if tool_call.function.name == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = str(e)
        if tool_call.function.name == "helper":
            function_result = helper(**json.loads(args),context = messages[1:])
        
        print('这是函数结果'+ function_result)
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        
        response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
        )



    
    return response

In [79]:
result = sql_querier('给我查一下最近三天有多少新的标')

这是函数参数{"query":"SELECT COUNT(*) FROM bids WHERE create_time \u003e= DATE_SUB(NOW(), INTERVAL 3 DAY);"}
这是函数结果1146 (42S02): Table 'sele.bids' doesn't exist
这是函数参数{"issue":"无法查询到最近三天的招标信息，因为数据库中不存在名为bids的表"}
这是函数结果首先，我建议对方Agent按照以下步骤进行问题排查和解决：

1. **确认需求**：
   - 确认一下需求是否明确，即“最近三天的招标信息”是否确实需要从数据库中名为`bids`的表中查询。

2. **检查数据库架构**：
   - 检查数据库中是否存在其他可能包含招标信息的表。
   - 查看数据库的文档或与数据库管理员沟通，确认是否有过表名变更或者数据结构更新。

3. **数据同步问题**：
   - 如果`bids`表应该存在，但当前不可见，可能是数据同步出现了问题。检查数据同步的任务计划，确认是否有错误或者同步延迟。

4. **权限问题**：
   - 确认Agent是否有权限访问名为`bids`的表。

以下是潜在的解决方案：

- **如果表名确实变更**：
  - 找到新的表名，并更新查询语句中的表名。
  - 如果是新创建的表，确认表结构后，更新查询逻辑。

- **如果是数据同步问题**：
  - 检查和重置数据同步作业，确保数据是最新的。
  - 如果有必要，手动触发数据同步过程。

- **如果表不存在且无法确定新表名**：
  - 检查是否有相关的数据库备份，从备份中查找`bids`表或者相关的表。
  - 如果没有备份，可以询问团队成员或者查阅相关文档来确定招标信息目前存储的位置。

- **权限问题**：
  - 联系数据库管理员，确认权限设置，并请求相应的访问权限。

- **其他可能**：
  - 如果以上都不适用，可能需要与负责数据维护的团队进行沟通，了解最近是否有数据相关的改动，或者是否有其他方式可以获得这些招标信息。

最后，建议对方Agent在解决问题后，更新相关的文档和通知团队成员，以避免未来出现同样的问题。希望这些建议能帮助到您！
这是函数参数{"query":"SELECT COUNT(*) F

In [150]:
from zhipuai import ZhipuAI
import json
import time


def sql_querier(input,context=None):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey

    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },
        ]

    role_setting = '''
    你是一个协助回答招标信息的助手。
    当你尝试回答用户问题，但是需要查询数据库时，你可以调用sql_query函数来查询数据库。
    
    1. 分析用户问题
    2. 调用sql_query函数，用show tables查看表格。
    3. 根据表名，选择“相关的表格”
    4. 调用sql_query函数，用show create table table_name 查看“相关的表格”的字段名
    5. 调用sql_query，查看相关字段名的前三个entry的内容。select 字段一，字段二 ... from table_name limited 3. 
    6. 如果相关表和字段信息能满足问题需要，就生成相应query，用sql_query函数查询相应数据
    7. 依据得到的数据回答用户的问题
    '''
    
    messages = [{"role": "system","content": role_setting}]
    if context:
        messages.append({"role": "user","content": '用户的问题是：' + input + '。上一次查询的结果是：' + context})
    else:
        messages.append({"role": "user","content": '那么开始，用户的问题是：' + input})
    
    response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    max_loop_count = 3
    loop_count = 0
    while loop_count < max_loop_count and response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        print('这是函数参数' + args)

        if tool_call.function.name == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = str(e)
        if tool_call.function.name == "helper":
            function_result = helper(**json.loads(args),context = messages[1:])
        
        print('这是函数结果'+ function_result)
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        
        response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
        )



    
    return response

In [90]:
result = sql_querier('给我查一下数据表里，最近三天有多少新的标')

print('最后的回答是：'+ result.choices[0].message.content)

最后的回答是：好的，为了回答您的问题，我需要查询数据库中标的信息。我将按照以下步骤进行：

1. 我需要查看数据库中的表格，以确定哪个表格包含标的信息。
2. 确定包含标的信息的表格后，我将检查该表格的结构，以确定如何查询最近三天的新的标。
3. 根据表格结构，我将生成一个查询，以统计最近三天内新标的数量。
4. 执行这个查询，并获取结果。
5. 最后，我将根据查询结果回答您的问题。

我现在开始执行这些步骤。首先，我将使用 `show tables` 命令查看数据库中的表格。请稍等片刻。


In [85]:
from zhipuai import ZhipuAI
import json
import logging
import time

def table_selector(input,context=None):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey

    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },
        {
        "type": "function",
        "function": {
            "name": "schema_analysis",
            "description": "该函数根据表格名字，去进一步回答问题",
            "parameters": {
                "type": "object",
                "properties": {
                    "tables": {
                        "description": "多个相关的表格名字",
                        "type": "string"
                    }
                },
                "required": ["tables"]
            },
            }
        },

        
        ]

    role_setting = '''
    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。
    你根据以下指引：
    1. 你先调用sql_query函数，用show tables查看表格。
    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。
    
    注：尽可能多的提供表格名字，以便schema_analysis函数更好的回答问题。
    '''
    
    messages = [{"role": "system","content": role_setting}]
    if context:
        messages.append({"role": "user","content": '用户的问题是：' + input + '。上次的反馈是：' + context})
    else:
        messages.append({"role": "user","content": '用户的问题是：' + input})
    
    max_retries = 3
    for i in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="glm-4", # 填写需要调用的模型名称
                messages=messages,
                tools=tools,
                tool_choice="auto",
                temperature=0.1,
            )
            break  # 如果没有错误，跳出重试循环
        except Exception as e:
            logging.error('在发送请求时发生错误: %s', e)
            if i < max_retries - 1:  # 如果不是最后一次尝试，等待一段时间后再次尝试
                time.sleep(2 ** i)
            else:  # 如果是最后一次尝试，抛出错误
                raise

    max_loop_count = 3
    loop_count = 0
    while loop_count < max_loop_count and response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        print('这是函数参数' + args)
        messages.append(response.choices[0].message.model_dump())
        print("这是添加了消息后的：")
        print(messages)
        if tool_call.function.name == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = str(e)
        if tool_call.function.name == "schema_analysis":
            print(messages)
            return args
        
        print('这是函数结果'+ function_result)
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        
        max_retries = 3
        for i in range(max_retries):
            try:
                response = client.chat.completions.create(
                    model="glm-4", # 填写需要调用的模型名称
                    messages=messages,
                    tools=tools,
                    tool_choice="auto",
                    temperature=0.1,
                )
                break  # 如果没有错误，跳出重试循环
            except Exception as e:
                logging.error('在发送请求时发生错误: %s', e)
                if i < max_retries - 1:  # 如果不是最后一次尝试，等待一段时间后再次尝试
                    time.sleep(2 ** i)
                else:  # 如果是最后一次尝试，抛出错误
                    raise
    print(messages)
    return response

In [86]:
result = table_selector("我想投房间工程的标，有哪些标可以投")

这是函数参数{"query":"show tables"}
这是添加了消息后的：
[{'role': 'system', 'content': '\n    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。\n    你根据以下指引：\n    1. 你先调用sql_query函数，用show tables查看表格。\n    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。\n    \n    注：尽可能多的提供表格名字，以便schema_analysis函数更好的回答问题。\n    '}, {'role': 'user', 'content': '用户的问题是：我想投房间工程的标，有哪些标可以投'}, {'content': None, 'role': 'assistant', 'tool_calls': [{'id': 'call_8611191038098362788', 'function': {'arguments': '{"query":"show tables"}', 'name': 'sql_query'}, 'type': 'function'}]}]
这是函数结果name 'sql_query' is not defined
这是函数参数{"tables":"room_project_bid"}
这是添加了消息后的：
[{'role': 'system', 'content': '\n    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。\n    你根据以下指引：\n    1. 你先调用sql_query函数，用show tables查看表格。\n    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。\n    \n    注：尽可能多的提供表格名字，以便schema_analysis函数更好的回答问题。\n    '}, {'role': 'user', 'content': '用户的问题是：我想投房间工程的标，有哪些标可以投'}, {'conte

In [155]:
result

Completion(model='glm-4', created=1714278496, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='用户的问题需要我们查找有关“房间工程”的标的信息。根据上一次的查询结果，我们需要寻找包含资质信息的表格，以便回答用户的问题。因此，我会先调用 `sql_query` 函数，使用 `show tables` 命令查看数据库中的所有表格，然后尝试找到包含资质信息的表格，并将这个表格的名字传递给 `schema_analysis` 函数。', role='assistant', tool_calls=None))], request_id='8605599231196891304', id='8605599231196891304', usage=CompletionUsage(prompt_tokens=329, completion_tokens=83, total_tokens=412))

In [136]:
from zhipuai import ZhipuAI
import json
import time

import logging

logging.basicConfig(level=logging.INFO)


def schema_analysis(input,context=None):
    client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc") # 请填写您自己的APIKey

    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },
        {
        "type": "function",
        "function": {
            "name": "answer_agent",
            "description": "根据table schema的信息，回答问题。",
            "parameters": {
                "type": "object",
                "properties": {
                    "schema": {
                        "description": "表格的schema信息",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },

        
        ]

    role_setting = '''
    你是一个处理小任务的agent。你根据“用户的问题”和“相关表格的提示”，查看表格结构，
    
    1. 使用 `sql_query` 函数，执行 `SHOW CREATE TABLE table_name` SQL命令来查看每一个“相关表格”的结构定义，包括所有字段名和表设置。
    2. 根据获得的字段名，再次调用 `sql_query` 函数，执行如 `SELECT 字段一, 字段二 FROM table_name LIMIT 3` 的查询，以预览每个字段的前三条记录，这有助于确认数据的实际内容和格式。
    3. 将表格名字，及其相应的所有表格的结构信息传递给 `answer_agent` 函数，让它来回答用户的问题。
    '''
    
    messages = [{"role": "system","content": role_setting}]
    if context:
        messages.append({"role": "user","content": '“用户问题”是：' + input + '。"相关表格"是：' + context})
    else:
        messages.append({"role": "user","content": '用户问题是：' + input})
    
    response = client.chat.completions.create(
        model="glm-4", # 填写需要调用的模型名称
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    max_loop_count = 5
    loop_count = 0
    while loop_count < max_loop_count and response.choices[0].message.tool_calls:
        tool_call = response.choices[0].message.tool_calls[0]
        args = tool_call.function.arguments
        print('这是函数参数: %s', args)
        
        if tool_call.function.name == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = str(e)
                print('在 sql_query 函数中发生错误: %s', e)
        if tool_call.function.name == "answer_agent":
            print('在 answer_agent 函数中，参数是: %s', args)
            return args
        
        print('这是函数结果: %s', function_result)
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call.id
        })
        
        max_retries = 3
        for i in range(max_retries):
            try:
                response = client.chat.completions.create(
                    model="glm-4", # 填写需要调用的模型名称
                    messages=messages,
                    tools=tools,
                    tool_choice="auto",
                )
                break  # 如果没有错误，跳出重试循环
            except Exception as e:
                logging.error('在发送请求时发生错误: %s', e)
                if i < max_retries - 1:  # 如果不是最后一次尝试，等待一段时间后再次尝试
                    time.sleep(2 ** i)
                else:  # 如果是最后一次尝试，抛出错误
                    raise

    return response

In [147]:
schema_result = schema_analysis("最近三天有多少新的招标公告",context=result)

INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


这是函数参数: %s {"query":"SHOW CREATE TABLE tender_index"}
这是函数结果: %s ('tender_index', "CREATE TABLE `tender_index` (\n  `id` int(11) NOT NULL AUTO_INCREMENT,\n  `title` varchar(255) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `publish_time` date DEFAULT NULL,\n  `province` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `source_platform` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `info_type` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `business_type` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `industry` varchar(100) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `detail_link` varchar(255) COLLATE utf8mb4_unicode_ci DEFAULT NULL,\n  `detail_info_fetched` tinyint(1) DEFAULT '0',\n  PRIMARY KEY (`id`)\n) ENGINE=InnoDB AUTO_INCREMENT=761 DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


这是函数参数: %s {"query":"SELECT COUNT(*) FROM tender_index WHERE publish_time \u003e= DATE_SUB(CURDATE(), INTERVAL 3 DAY)"}
这是函数结果: %s (0,)


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


In [148]:
print(schema_result)

model='glm-4' created=1714278156 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='根据最近的招标公告表格`tender_index`的结构信息，我们可以看到该表格包含招标公告的各种信息，如标题、发布时间、省份、来源平台等。然而，根据我们的查询结果，最近三天内似乎没有新的招标公告发布。', role='assistant', tool_calls=None))] request_id='8605595073668418439' id='8605595073668418439' usage=CompletionUsage(prompt_tokens=647, completion_tokens=53, total_tokens=700)


In [ ]:
{
"type": "function",
"function": {
    "name": "table_selector",
    "description": "依据用户问题和schema_analysis的反馈，选择最合适的表格。",
    "parameters": {
        "type": "object",
        "properties": {
            "feedback": {
                "description": "反馈还需要什么表格，或者表格不够用的信息",
                "type": "string"
            }
        },
        "required": ["feedback"]
    },
    }
},
{
"type": "function",
"function": {
    "name": "answer_agent",
    "description": "根据query，回答用户的问题。",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                "type": "string"
            }
        },
        "required": ["query"]
    },
    }
},


In [185]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc")

tools = []

response_zhipu = client.chat.completions.create(
    model="glm-4",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    temperature=0.1,
)


INFO:httpx:HTTP Request: POST https://open.bigmodel.cn/api/paas/v4/chat/completions "HTTP/1.1 200 OK"


In [ ]:
import dashscope

dashscope.api_key="sk-cbcc1fb859b1456885a270eecbec6369"

response_qwen = dashscope.Generation.call(
    model='qwen-turbo',
    messages=messages,
    tools=tools,
    seed=random.randint(1, 10000),
    result_format='message'  
)


In [6]:
class AIModel:
    def __init__(self, api_key, model, temperature=0.9,tools=None):
        self.api_key = api_key
        self.model = model
        self.tools = tools
        self.temperature = temperature
        self.message = None

    def call(self, messages):
        result = self.get_result(messages)
        self.process_result(result)
        return self

    def get_result(self, messages):
        raise NotImplementedError("Subclasses must implement this method")

    def process_result(self, result):
        self.message = self.get_message(result)
        self.usage = self.get_usage(result)

    def get_message(self, result):
        raise NotImplementedError("Subclasses must implement this method")

    def get_usage(self, result):
        raise NotImplementedError("Subclasses must implement this method")


class ZhipuModel(AIModel):
    def __init__(self, api_key, model, temperature=0.9,tools=None):
        super().__init__(api_key, model, temperature,tools)
        from zhipuai import ZhipuAI
        self.client = ZhipuAI(api_key=self.api_key)

    def get_result(self, messages):
        return self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools = self.tools,
            tool_choice="auto",
            temperature=self.temperature,
        )

    def get_message(self, result):
        message = {
            'role': result.choices[0].message.role,
            'content': result.choices[0].message.content,
        }
        if result.choices[0].message.tool_calls is not None:
            message['tool_calls'] = result.choices[0].message.tool_calls
        return message
    
    def get_usage(self, result):
        return {
            "input_tokens": result.usage.prompt_tokens,
            "output_tokens": result.usage.completion_tokens,
            "total_tokens": result.usage.total_tokens
        }


class QwenModel(AIModel):
    def __init__(self, api_key, model, temperature=0.9,tools=None):
        super().__init__(api_key, model, temperature, tools)
        import dashscope
        dashscope.api_key = self.api_key

    def get_result(self, messages):
        import dashscope
        import random
        return dashscope.Generation.call(
            model=self.model,
            messages=messages,
            tools=self.tools,
            seed=random.randint(1, 10000),
            temperature=self.temperature,
            result_format='message'  
        )

    def get_message(self, result):
        return result.output.choices[0].message

    def get_usage(self, result):
        return {
            "input_tokens": result.usage.input_tokens,
            "output_tokens": result.usage.output_tokens,
            "total_tokens": result.usage.total_tokens
        }

In [136]:
# llm = QwenModel(model='qwen-turbo', api_key="sk-cbcc1fb859b1456885a270eecbec6369")

llm = ZhipuModel(model='glm-4', api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc")


In [139]:
tools = [
    {
    "type": "function",
    "function": {
        "name": "sql_query",
        "description": "执行 MySQL SQL 查询并返回结果。",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                    "type": "string"
                }
            },
            "required": ["query"]
        },
        }
    },
    {
    "type": "function",
    "function": {
        "name": "schema_analysis",
        "description": "该函数根据表格名字，去进一步回答问题",
        "parameters": {
            "type": "object",
            "properties": {
                "tables": {
                    "description": "多个相关的表格名字",
                    "type": "string"
                }
            },
            "required": ["tables"]
        },
        }
    },


    ]


llm = ZhipuModel(model='glm-4', api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc",tools=tools)
# llm = QwenModel(model='qwen-turbo', api_key="sk-cbcc1fb859b1456885a270eecbec6369",tools=tools)


role_setting = '''
    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。
    你根据以下指引：
    1. 你先调用sql_query函数，用show tables查看表格。
    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。
    
    '''
    
messages = [{"role": "system","content": role_setting}]

messages.append({"role": "user","content": '你好，我准备问个问题'})

response = llm.call(messages)


In [140]:
response.message

{'role': 'assistant', 'content': '好的，请问你要问什么问题呢？'}

In [94]:
type(response.message["tool_calls"])

list

In [145]:
from zhipuai import ZhipuAI
import json
import logging
import time

def table_selector(input,context=None):

    tools = [
        {
        "type": "function",
        "function": {
            "name": "sql_query",
            "description": "执行 MySQL SQL 查询并返回结果。",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "description": "MySQL的命令语句 SQL query。此语句必须符合 SQL 语法规范，确保可以正确执行以获取预期的数据库信息。",
                        "type": "string"
                    }
                },
                "required": ["query"]
            },
            }
        },
        {
        "type": "function",
        "function": {
            "name": "schema_analysis",
            "description": "该函数根据表格名字，去进一步回答问题",
            "parameters": {
                "type": "object",
                "properties": {
                    "tables": {
                        "description": "多个相关的表格名字",
                        "type": "string"
                    }
                },
                "required": ["tables"]
            },
            }
        },
        ]

    # llm = ZhipuModel(model='glm-4', api_key="fa1467996b5f2018bcf1f49f1643cf43.908Q1sveABVE1WJc",tools=tools)
    llm = QwenModel(model='qwen-turbo', api_key="sk-cbcc1fb859b1456885a270eecbec6369",tools=tools)

    role_setting = '''
    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。
    你根据以下指引：
    1. 你先调用sql_query函数，用show tables查看表格。
    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。
    '''
    
    messages = [{"role": "system","content": role_setting}]
    if context:
        messages.append({"role": "user","content": '用户的问题是：' + input + '。上次的反馈是：' + context})
    else:
        messages.append({"role": "user","content": '用户的问题是：' + input})
    
    max_retries = 3
    for i in range(max_retries):
        try:
            response = llm.call(messages)
            break  # 如果没有错误，跳出重试循环
        except Exception as e:
            logging.error('在发送请求时发生错误: %s', e)
            if i < max_retries - 1:  # 如果不是最后一次尝试，等待一段时间后再次尝试
                time.sleep(2 ** i)
            else:  # 如果是最后一次尝试，抛出错误
                raise

    max_loop_count = 3
    loop_count = 0
    while loop_count < max_loop_count and ("tool_calls" in response.message):
        tool_call = response.message["tool_calls"][0]
        args = tool_call["function"]["arguments"]
        print('这是函数参数' + args)
        messages.append(response.message)
        print("这是添加了消息后的：")
        print(messages)
        if tool_call["function"]["name"] == "sql_query":
            try:
                function_result = sql_query(**json.loads(args))
            except Exception as e:
                function_result = str(e)
        if tool_call["function"]["name"] == "schema_analysis":
            print(messages)
            return args
        
        print('这是函数结果'+ function_result)
        messages.append({
            "role": "tool",
            "content": f"{json.dumps(function_result)}",
            "tool_call_id":tool_call['id']
        })
        
        max_retries = 3
        for i in range(max_retries):
            try:
                response = llm.call(messages)                
                break  # 如果没有错误，跳出重试循环
            except Exception as e:
                logging.error('在发送请求时发生错误: %s', e)
                if i < max_retries - 1:  # 如果不是最后一次尝试，等待一段时间后再次尝试
                    time.sleep(2 ** i)
                else:  # 如果是最后一次尝试，抛出错误
                    raise
    print(messages)
    return response

In [146]:
result = table_selector("我想投房屋建筑工程项目的标，有哪些标可以投")

这是函数参数{"query": "SHOW TABLES LIKE '%project%' OR LIKE '%construction%'"}
这是添加了消息后的：
[{'role': 'system', 'content': '\n    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。\n    你根据以下指引：\n    1. 你先调用sql_query函数，用show tables查看表格。\n    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis函数，让它来完成问题的回答。\n    '}, {'role': 'user', 'content': '用户的问题是：我想投房屋建筑工程项目的标，有哪些标可以投'}, Message({'role': 'assistant', 'content': '要回答这个问题，我们需要知道具体的数据库结构和其中包含的信息。但是，如果我们假设有一个名为`project_bids`或者`construction_projects`的表，其中可能包含了关于工程项目（如类型、阶段、招标信息等）的数据，我们可以尝试查询这些信息。首先，让我们通过SQL查询来查看有哪些与房屋建筑工程项目相关的表：', 'tool_calls': [{'function': {'name': 'sql_query', 'arguments': '{"query": "SHOW TABLES LIKE \'%project%\' OR LIKE \'%construction%\'"}'}, 'id': '', 'type': 'function'}]})]
这是函数结果name 'sql_query' is not defined
[{'role': 'system', 'content': '\n    你分析用户的问题，确定用户问题会包含在什么表里，并告诉schema_analysis函数，让它来完成问题的回答。\n    你根据以下指引：\n    1. 你先调用sql_query函数，用show tables查看表格。\n    2. 依据表名，以及反馈，选择“相关的表格”，然后将相关的表格名字传给schema_analysis

In [148]:
result.message

Message({'role': 'assistant', 'content': '对不起，我似乎无法直接访问数据库来执行SQL查询。但通常情况下，如果存在与房屋建筑工程项目相关的表，可能会有`project_bids`、`construction_projects`、`bidding_documents`等。请提供这些表之一的名字，或者告诉我这些表在您的数据库中的具体名称，我可以帮您分析可能的投标类别。'})

In [9]:
def get_ai_response(user_input):
    messages = [
        {
            "role": "system",
            "content": "你是一个尝试有趣回答的助手"
        },
        {
            "role": "user",
            "content": user_input
        }
    ]
    # model = ZhipuModel(api_key=api_key, model="glm-4", temperature=0.9,tools=sql_tools)
    model = QwenModel(api_key="sk-cbcc1fb859b1456885a270eecbec6369", model="qwen-turbo", temperature=0.9)
    
    model.call(messages)
    answer = model.message['content']
    return answer

In [10]:
answer = get_ai_response('whats up!')

In [11]:
answer

'Oh, just hanging around, spreading some good vibes and waiting for someone like you to ask an interesting question. How can I brighten your day?'